In [6]:
# 本代码由可视化策略环境自动生成 2021年8月4日 16:42
# 本代码单元只能在可视化模式下编辑。您也可以拷贝代码，粘贴到新建的代码单元或者策略，然后修改。


# 回测引擎：初始化函数，只执行一次
def m3_initialize_bigquant_run(context):
    # 手续费设置
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5)) 

# 回测引擎：每日数据处理函数，每天执行一次
def m3_handle_data_bigquant_run(context, data):
    date = data.current_dt.strftime('%Y-%m-%d')
    # 通过positions对象，使用列表生成式的方法获取目前持仓的股票列表
    stock_hold_now = [equity.symbol for equity in context.portfolio.positions]

# 下面注释代码为大盘择时，大家可以根据自己情况选择使用    
#     bench_mark_ma_short = D.history_data(['000300.SHA'],'2006-01-01', date, ['close'])['close'].tail(10).mean()
#     bench_mark_ma_long = D.history_data(['000300.SHA'],'2006-01-01', date, ['close'])['close'].tail(40).mean()
#     if bench_mark_ma_short < bench_mark_ma_long and len(stock_hold_now) > 0:
#         for j in stock_hold_now:
#             context.order_target_percent(context.symbol(j), 0)
            
    # 按月调仓
    if context.trading_day_index % 21 != 0:
        return 
    # 根据日期获取调仓需要买入的股票的列表
    stock_to_buy = context.daily_buy_stock.ix[date]
    # 通过positions对象，使用列表生成式的方法获取目前持仓的股票列表
    stock_hold_now = [equity.symbol for equity in context.portfolio.positions]
    # 继续持有的股票：调仓时，如果买入的股票已经存在于目前的持仓里，那么应继续持有
    no_need_to_sell = [i for i in stock_hold_now if i in stock_to_buy]
    # 需要卖出的股票
    stock_to_sell = [i for i in stock_hold_now if i not in no_need_to_sell]
  
    # 卖出
    for stock in stock_to_sell:
        # 如果该股票停牌，则没法成交。因此需要用can_trade方法检查下该股票的状态
        # 如果返回真值，则可以正常下单，否则会出错
        # 因为stock是字符串格式，我们用symbol方法将其转化成平台可以接受的形式：Equity格式
        if data.can_trade(context.symbol(stock)):
            # order_target_percent是平台的一个下单接口，表明下单使得该股票的权重为0，
            #   即卖出全部股票，可参考回测文档
            context.order_target_percent(context.symbol(stock), 0)
    
    # 如果当天没有买入的股票，就返回
    if len(stock_to_buy) == 0:
        return

    # 等权重买入 
    weight =  1 / len(stock_to_buy)
    
    # 买入
    for stock in stock_to_buy:
        if data.can_trade(context.symbol(stock)):
            # 下单使得某只股票的持仓权重达到weight，因为
            # weight大于0,因此是等权重买入
            context.order_target_percent(context.symbol(stock), weight)
# 回测引擎：准备数据，只执行一次
def m3_prepare_bigquant_run(context):
    instruments =context.instruments
    start_date = context.start_date 
    end_date = context.end_date
    price_data = D.history_data(instruments, start_date, end_date, fields=['close'])
    ret_data = price_data.groupby('instrument').apply(calcu_ret)
    ret_data.reset_index(inplace=True, drop=True)
    ret_data['date'] = ret_data['date'].map(lambda x:x.strftime('%Y-%m-%d'))
    daily_buy_stock = ret_data.groupby('date').apply(seek_stock)
    context.daily_buy_stock = daily_buy_stock
    
# 计算不同期限的收益率
def calcu_ret(df):
    df = df.sort_values('date')
    for i in [21,63,126,252]:
        df['ret_%s'%i] = df['close']/df['close'].shift(i)-1 
    return df

# 计算出收益率得分
def seek_stock(df):
    for j in ['ret_21','ret_63','ret_126','ret_252']:
        df['%s'%j] = df['%s'%j].rank(ascending=True) 
    df['score'] = 0.3*df['ret_21']+0.3*df['ret_63']+0.2*df['ret_126']+0.2*df['ret_252']
    result = df.sort_values('score', ascending=False)
    return list(result.instrument)[:20] 
# 回测引擎：每个单位时间开始前调用一次，即每日开盘前调用一次。
def m3_before_trading_start_bigquant_run(context, data):
    pass


m1 = M.instruments.v2(
    start_date='2016-01-01',
    end_date='2018-03-22',
    market='CN_FUND',
    instrument_list='',
    max_count=0
)

m3 = M.trade.v4(
    instruments=m1.data,
    start_date='',
    end_date='',
    initialize=m3_initialize_bigquant_run,
    handle_data=m3_handle_data_bigquant_run,
    prepare=m3_prepare_bigquant_run,
    before_trading_start=m3_before_trading_start_bigquant_run,
    volume_limit=0.025,
    order_price_field_buy='open',
    order_price_field_sell='open',
    capital_base=1000000,
    auto_cancel_non_tradable_orders=True,
    data_frequency='daily',
    price_type='后复权',
    product_type='股票',
    plot_charts=True,
    backtest_only=False,
    benchmark='000300.SHA'
)


[2018-03-22 17:52:59.648426] INFO: bigquant: instruments.v2 开始运行..
[2018-03-22 17:52:59.652457] INFO: bigquant: 命中缓存
[2018-03-22 17:52:59.653852] INFO: bigquant: instruments.v2 运行完成[0.005476s].
[2018-03-22 17:52:59.707649] INFO: bigquant: backtest.v7 开始运行..
[2018-03-22 17:53:08.998792] INFO: algo: set price type:backward_adjusted
[2018-03-22 17:53:22.794980] INFO: Performance: Simulated 541 trading days out of 541.
[2018-03-22 17:53:22.797176] INFO: Performance: first open: 2016-01-04 01:30:00+00:00
[2018-03-22 17:53:22.798392] INFO: Performance: last close: 2018-03-22 07:00:00+00:00


[2018-03-22 17:53:28.045433] INFO: bigquant: backtest.v7 运行完成[28.337734s].
